In [15]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Download latest version
path = kagglehub.dataset_download("netflix-inc/netflix-prize-data")

print("Path to dataset folder:", path)

Path to dataset folder: C:\Users\Nirmal Choyal\.cache\kagglehub\datasets\netflix-inc\netflix-prize-data\versions\2


In [29]:
import os
import csv
import pandas as pd

In [ ]:
# List files in the dataset folder
print("\nFiles inside dataset folder:")
print(os.listdir(path))


Files inside dataset folder:
['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt', 'movie_titles.csv', 'probe.txt', 'qualifying.txt', 'README']


In [ ]:
# Read and display the first 5 lines of combined_data_1.txt 
file_path = os.path.join(path, "combined_data_1.txt")
with open(file_path, "r") as f:
    for _ in range(5):
        print(f.readline().strip())

1:
1488844,3,2005-09-06
822109,5,2005-05-13
885013,4,2005-10-19
30878,4,2005-12-26


In [ ]:
# Read movie titles from movie_titles.csv
path1 = os.path.join(path, "movie_titles.csv")

rows = []
with open(path1, "r", encoding="latin-1") as f:
    reader = csv.reader(f)
    for row in reader:
        movie_id = int(row[0])
        year = row[1] if row[1] != "" else None
        title = ",".join(row[2:])   # join remaining parts safely
        rows.append([movie_id, year, title])

movies = pd.DataFrame(rows, columns=["MovieID", "Year", "Title"])
movies.head()


,MovieID,Year,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW
